<a href="https://colab.research.google.com/github/dwwatson1/coronavirus_pandemic_playbook/blob/main/script/covid_cases_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import io

In [ ]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
link = 'https://drive.google.com/file/d/1MqByhO2M424pdBSLD8mYBoJBr7hg0pNe/view?usp=sharing' # The shareable link

In [ ]:
id = '1MqByhO2M424pdBSLD8mYBoJBr7hg0pNe'
print (id)

1MqByhO2M424pdBSLD8mYBoJBr7hg0pNe


In [ ]:
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('covid_cases.csv')  
covid_df = pd.read_csv('covid_cases.csv')

In [ ]:
chunk_data = [chunk for chunk in data_chunk]
covid_df = pd.concat(chunk_data)

In [ ]:
covid_df = pd.read_csv('covid_cases.csv')

In [ ]:
covid_df = covid_df[(covid_df['res_state'] == 'OK') | (covid_df['res_state'] == 'TN') | (covid_df['res_state'] == 'WI') | (covid_df['res_state'] == 'UT') | (covid_df['res_state'] == 'CT')]

In [ ]:
covid_df

,age_group,sex,region,pop_density,median_income,airport_area,policy_prevention,has_covid
535,0,1,2,242,67355,0.03,79,1
536,0,0,2,242,67355,0.03,79,1
537,2,1,2,242,67355,0.03,79,1
538,2,0,2,242,67355,0.03,79,1
539,0,1,2,242,67355,0.03,79,1
...,...,...,...,...,...,...,...,...
158681197,2,1,2,242,67355,0.03,79,1
158681198,1,1,2,242,67355,0.03,79,1
158681199,3,1,2,242,67355,0.03,79,1
158681200,3,0,2,242,67355,0.03,79,1


In [ ]:
covid_df = covid_df.drop(columns=['res_state', 'race'])

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [ ]:
covid_cat = covid_df.dtypes[covid_df.dtypes == "object"].index.tolist()

from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse=False)
encode_df = pd.DataFrame(enc.fit_transform(covid_df[covid_cat]))

encode_df.columns = enc.get_feature_names(covid_cat)

In [ ]:
covid_df_encode = covid_df.merge(encode_df, left_index=True, right_index=True)
covid_df_encode = covid_df.drop(covid_cat,1)

In [ ]:
y = covid_df["has_covid"]
X = covid_df.drop(columns="has_covid")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

scaler = StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
rf_model = RandomForestClassifier(n_estimators=128, random_state=1)

rf_model = rf_model.fit(X_train_scaled, y_train)

In [ ]:
predictions = rf_model.predict(X_test_scaled)

In [ ]:
cm = confusion_matrix(y_test, predictions)

cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns = ["Predicted 0", "Predicted 1"])

acc_score = accuracy_score(y_test, predictions)

In [ ]:
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score: {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,0,454133
Actual 1,0,6757776


Accuracy Score: 0.9370301261427453
Classification Report


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00    454133
           1       0.94      1.00      0.97   6757776

    accuracy                           0.94   7211909
   macro avg       0.47      0.50      0.48   7211909
weighted avg       0.88      0.94      0.91   7211909



In [ ]:
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.4454270888902362, 'age_group'),
 (0.1578296255050766, 'airport_area'),
 (0.15569557431751424, 'policy_prevention'),
 (0.13141942612920784, 'pop_density'),
 (0.06267726502633213, 'median_income'),
 (0.027735106350305783, 'region'),
 (0.019215913781327202, 'sex')]